In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

from config.experiment_config import ExperimentConfig

In [3]:
data_df = None
for file in os.listdir(ExperimentConfig.PATH_FEATURES):
    if file not in ("ipos.xlsx", "spx_interim.xlsx"):
        print(file)
        if file not in (
            "bbb_credit.xlsx",
            "global_bonds_total_r.xlsx",
            "10y.xlsx",
            "default_yield.xlsx",
            "inflation_total_r.xlsx",
            "libor3m.xlsx",
            "indust_prod.xlsx",
            "sentiment.xlsx",
        ):
            interim_df = pd.read_excel(
                ExperimentConfig.PATH_FEATURES / file,
                parse_dates=True,
                date_format="%Y-%m-%d",
                skiprows=6,
            )
        else:
            interim_df = pd.read_excel(
                ExperimentConfig.PATH_FEATURES / file,
                parse_dates=True,
                date_format="%Y-%m-%d",
                skiprows=5,
            )

        # print(interim_df.head())
        interim_df = interim_df.rename(
            columns={
                "PX_LAST": f"{file.split('.xlsx')[0]}",
                "Date": "date",
                "PX_BID": f"{file.split('.xlsx')[0]}_bid",
            }
        )
        if "CHG_PCT_1D" in interim_df.columns:
            interim_df = interim_df.drop(columns=["CHG_PCT_1D"])
        interim_df["date"] = pd.to_datetime(interim_df["date"], format="%d.%m.%Y")
        interim_df = interim_df.set_index("date")
        interim_df = interim_df.sort_index()

        if data_df is None:
            data_df = interim_df
        else:
            data_df = pd.merge_asof(
                data_df,
                interim_df,
                left_index=True,
                right_index=True,
                tolerance=pd.Timedelta(days=1),
                direction="backward",
            )
data_df

10y.xlsx
bbb_credit.xlsx
default_yield.xlsx
erp.xlsx
est_pe.xlsx
global_bonds_total_r.xlsx
indust_prod.xlsx
inflation_swap.xlsx
inflation_total_r.xlsx
ishares_pure_credit.xlsx
libor3m.xlsx
sentiment.xlsx
si.xlsx
spx.xlsx
spx_add.xlsx
tail.xlsx
ts.xlsx
world_etf.xlsx


,10y,bbb_credit,default_yield,erp,erp_bid,est_pe,BEST_EPS,global_bonds_total_r,indust_prod,inflation_swap,...,NET_DEBT_TO_EBITDA,spx_add,spx_add_bid,CORR_COEF,SHORT_AND_LONG_TERM_DEBT,tail,tail_bid,ts,world_etf,world_etf_bid
date,,,,,,,,,,,,,,,,,,,,,
1962-01-02,4.0600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,70.96,NaN,0.908820,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-03,4.0300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,71.13,NaN,0.908825,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-04,3.9900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,70.64,NaN,0.908752,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-05,4.0220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,69.66,NaN,0.908491,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-08,4.0320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,69.12,NaN,0.908467,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-22,4.6109,142.72,2.63,293.42,NaN,6086.37,238.3995,464.1360,NaN,2.5397,...,1.4806,6086.37,6017.21,0.853975,1409.016,530.86,NaN,30.919,160.89,160.84
2025-01-23,4.6436,140.88,2.63,296.42,NaN,6118.71,238.2551,463.2555,NaN,2.5770,...,1.4806,6118.71,6052.69,0.853795,1409.016,533.57,NaN,35.033,161.87,161.83
2025-01-24,4.6214,145.86,2.61,294.73,NaN,6101.24,238.0381,465.5408,NaN,2.5557,...,1.4806,6101.24,6055.63,0.854142,1409.016,532.10,NaN,34.959,161.74,161.73


In [4]:
data_df = data_df.dropna(subset=["spx"])
data_df["spread"] = data_df["spx"] - data_df["spx_bid"]
data_df["spx"] = data_df["spx"] / data_df["spx"].shift(1) - 1
data_df

C:\Users\Ashnv\AppData\Local\Temp\ipykernel_8452\3357174383.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["spread"] = data_df["spx"] - data_df["spx_bid"]
C:\Users\Ashnv\AppData\Local\Temp\ipykernel_8452\3357174383.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["spx"] = data_df["spx"] / data_df["spx"].shift(1) - 1


,10y,bbb_credit,default_yield,erp,erp_bid,est_pe,BEST_EPS,global_bonds_total_r,indust_prod,inflation_swap,...,spx_add,spx_add_bid,CORR_COEF,SHORT_AND_LONG_TERM_DEBT,tail,tail_bid,ts,world_etf,world_etf_bid,spread
date,,,,,,,,,,,,,,,,,,,,,
1962-01-02,4.0600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,70.96,NaN,0.908820,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-03,4.0300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,71.13,NaN,0.908825,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-04,3.9900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,70.64,NaN,0.908752,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-05,4.0220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69.66,NaN,0.908491,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-08,4.0320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69.12,NaN,0.908467,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-22,4.6109,142.72,2.63,293.42,NaN,6086.37,238.3995,464.1360,NaN,2.5397,...,6086.37,6017.21,0.853975,1409.016,530.86,NaN,30.919,160.89,160.84,69.16
2025-01-23,4.6436,140.88,2.63,296.42,NaN,6118.71,238.2551,463.2555,NaN,2.5770,...,6118.71,6052.69,0.853795,1409.016,533.57,NaN,35.033,161.87,161.83,66.02
2025-01-24,4.6214,145.86,2.61,294.73,NaN,6101.24,238.0381,465.5408,NaN,2.5557,...,6101.24,6055.63,0.854142,1409.016,532.10,NaN,34.959,161.74,161.73,45.61


In [5]:
data_df["rate"] = data_df["10y"] / 100 / 365
data_df["day_count"] = data_df.index.diff().days
data_df.iloc[0, 1] = 1
data_df["acc_rate"] = data_df["rate"] * data_df["day_count"]

C:\Users\Ashnv\AppData\Local\Temp\ipykernel_8452\790716145.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["rate"] = data_df["10y"] / 100 / 365
C:\Users\Ashnv\AppData\Local\Temp\ipykernel_8452\790716145.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["day_count"] = data_df.index.diff().days
C:\Users\Ashnv\AppData\Local\Temp\ipykernel_8452\790716145.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [6]:
data_df

,10y,bbb_credit,default_yield,erp,erp_bid,est_pe,BEST_EPS,global_bonds_total_r,indust_prod,inflation_swap,...,SHORT_AND_LONG_TERM_DEBT,tail,tail_bid,ts,world_etf,world_etf_bid,spread,rate,day_count,acc_rate
date,,,,,,,,,,,,,,,,,,,,,
1962-01-02,4.0600,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000111,NaN,NaN
1962-01-03,4.0300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000110,1.0,0.000110
1962-01-04,3.9900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000109,1.0,0.000109
1962-01-05,4.0220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000110,1.0,0.000110
1962-01-08,4.0320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000110,3.0,0.000331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-22,4.6109,142.72,2.63,293.42,NaN,6086.37,238.3995,464.1360,NaN,2.5397,...,1409.016,530.86,NaN,30.919,160.89,160.84,69.16,0.000126,1.0,0.000126
2025-01-23,4.6436,140.88,2.63,296.42,NaN,6118.71,238.2551,463.2555,NaN,2.5770,...,1409.016,533.57,NaN,35.033,161.87,161.83,66.02,0.000127,1.0,0.000127
2025-01-24,4.6214,145.86,2.61,294.73,NaN,6101.24,238.0381,465.5408,NaN,2.5557,...,1409.016,532.10,NaN,34.959,161.74,161.73,45.61,0.000127,1.0,0.000127


In [7]:
data_df = data_df.drop(["spx_bid"], axis=1)
data_df = data_df.ffill()
# data_df = data_df.dropna(axis=1)
data_df

,10y,bbb_credit,default_yield,erp,erp_bid,est_pe,BEST_EPS,global_bonds_total_r,indust_prod,inflation_swap,...,SHORT_AND_LONG_TERM_DEBT,tail,tail_bid,ts,world_etf,world_etf_bid,spread,rate,day_count,acc_rate
date,,,,,,,,,,,,,,,,,,,,,
1962-01-02,4.0600,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000111,NaN,NaN
1962-01-03,4.0300,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000110,1.0,0.000110
1962-01-04,3.9900,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000109,1.0,0.000109
1962-01-05,4.0220,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000110,1.0,0.000110
1962-01-08,4.0320,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000110,3.0,0.000331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-22,4.6109,142.72,2.63,293.42,0.0,6086.37,238.3995,464.1360,0.92,2.5397,...,1409.016,530.86,NaN,30.919,160.89,160.84,69.16,0.000126,1.0,0.000126
2025-01-23,4.6436,140.88,2.63,296.42,0.0,6118.71,238.2551,463.2555,0.92,2.5770,...,1409.016,533.57,NaN,35.033,161.87,161.83,66.02,0.000127,1.0,0.000127
2025-01-24,4.6214,145.86,2.61,294.73,0.0,6101.24,238.0381,465.5408,0.92,2.5557,...,1409.016,532.10,NaN,34.959,161.74,161.73,45.61,0.000127,1.0,0.000127


In [8]:
features = []
FIRST_VALID_DATE = pd.Timestamp("2004-03-19")
for column in data_df.columns:
    first_valid_index = data_df[column].first_valid_index()
    print(f"{column} -> {first_valid_index}")
    if first_valid_index is not None and first_valid_index <= FIRST_VALID_DATE:
        features.append(column)

10y -> 1962-01-02 00:00:00
bbb_credit -> 1962-01-02 00:00:00
default_yield -> 1987-01-30 00:00:00
erp -> 1997-09-09 00:00:00
erp_bid -> 2023-01-18 00:00:00
est_pe -> 1979-02-16 00:00:00
BEST_EPS -> 1990-03-30 00:00:00
global_bonds_total_r -> 1990-01-02 00:00:00
indust_prod -> 1962-01-31 00:00:00
inflation_swap -> 2004-07-21 00:00:00
inflation_total_r -> 1997-10-31 00:00:00
ishares_pure_credit -> 2015-09-11 00:00:00
ishares_pure_credit_bid -> 2015-09-11 00:00:00
libor3m -> 1984-12-06 00:00:00
sentiment -> 1987-07-23 00:00:00
si -> 2013-02-05 00:00:00
si_bid -> 2020-06-30 00:00:00
spx -> 1962-01-03 00:00:00
DVD_PAYOUT_RATIO -> 1991-02-04 00:00:00
DVD_SH_LAST -> 1980-02-29 00:00:00
NET_AGGTE_DVD_YLD -> 1990-01-31 00:00:00
PE_RATIO -> 1990-05-10 00:00:00
BEST_CAPEX -> 2007-12-18 00:00:00
PX_TO_BOOK_RATIO -> 1990-05-10 00:00:00
NET_DEBT_TO_EBITDA -> 1991-02-04 00:00:00
spx_add -> 1962-01-02 00:00:00
spx_add_bid -> 1998-03-02 00:00:00
CORR_COEF -> 1962-01-02 00:00:00
SHORT_AND_LONG_TERM_DEBT

In [9]:
data_df = data_df[features]
data_df = data_df.loc[FIRST_VALID_DATE:]
data_df

,10y,bbb_credit,default_yield,erp,est_pe,BEST_EPS,global_bonds_total_r,indust_prod,inflation_total_r,libor3m,...,spx_add,spx_add_bid,CORR_COEF,SHORT_AND_LONG_TERM_DEBT,tail,ts,spread,rate,day_count,acc_rate
date,,,,,,,,,,,,,,,,,,,,,
2004-03-19,3.7710,232.90,3.44,78.67,1109.78,63.9681,298.3774,0.62,190.9748,1.11000,...,1109.78,1109.56,0.977094,841.4458,100.00,225.576,0.22,0.000103,1.0,0.000103
2004-03-22,3.7126,234.74,3.52,77.14,1095.40,63.9851,299.2598,0.62,191.9312,1.11000,...,1095.40,1094.91,0.977192,841.4458,99.10,223.155,0.49,0.000102,3.0,0.000305
2004-03-23,3.6901,236.99,3.57,76.92,1093.95,64.0646,298.7925,0.62,191.7853,1.11000,...,1093.95,1093.82,0.977133,841.4458,98.92,222.618,0.13,0.000101,1.0,0.000101
2004-03-24,3.7069,235.31,3.55,76.92,1091.33,64.0777,298.4857,0.62,191.4800,1.11000,...,1091.33,1091.06,0.977131,841.4458,98.70,224.636,0.27,0.000102,1.0,0.000102
2004-03-25,3.7368,235.32,3.51,78.21,1109.19,64.1440,298.1168,0.62,190.6442,1.11000,...,1109.19,1108.62,0.977195,843.2527,99.82,226.068,0.57,0.000102,1.0,0.000102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-22,4.6109,142.72,2.63,293.42,6086.37,238.3995,464.1360,0.92,323.3708,4.85372,...,6086.37,6017.21,0.853975,1409.0160,530.86,30.919,69.16,0.000126,1.0,0.000126
2025-01-23,4.6436,140.88,2.63,296.42,6118.71,238.2551,463.2555,0.92,323.4737,4.85372,...,6118.71,6052.69,0.853795,1409.0160,533.57,35.033,66.02,0.000127,1.0,0.000127
2025-01-24,4.6214,145.86,2.61,294.73,6101.24,238.0381,465.5408,0.92,325.4554,4.85372,...,6101.24,6055.63,0.854142,1409.0160,532.10,34.959,45.61,0.000127,1.0,0.000127


In [10]:
data_df.isna().any().any()

False

In [11]:
data_df[data_df.columns.difference(["rate", "day_count"])].to_csv(
    ExperimentConfig.PATH_OUTPUT / ExperimentConfig.INITIAL_FEATURES_FILENAME
)